<a href="https://colab.research.google.com/github/smwaingeni-ai/AgriX-AfricaDeepTech2025/blob/main/prototype/main_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [54]:
# STEP 1: Install required packages
!pip install opencv-python-headless --quiet

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os
from google.colab import files
import shutil

In [55]:
# STEP 2: Clone your repo (remove old first if needed)
!rm -rf AgriX-AfricaDeepTech2025
!git clone https://github.com/smwaingeni-ai/AgriX-AfricaDeepTech2025.git
%cd AgriX-AfricaDeepTech2025


Cloning into 'AgriX-AfricaDeepTech2025'...
remote: Enumerating objects: 662, done.
remote: Counting objects: 100% (391/391), done.
remote: Compressing objects: 100% (312/312), done.
remote: Total 662 (delta 231), reused 78 (delta 78), pack-reused 271 (from 2)
Receiving objects: 100% (662/662), 24.58 MiB | 13.68 MiB/s, done.
Resolving deltas: 100% (270/270), done.
/content/AgriX-AfricaDeepTech2025/AgriX-AfricaDeepTech2025/AgriX-AfricaDeepTech2025/AgriX-AfricaDeepTech2025/AgriX-AfricaDeepTech2025/AgriX-AfricaDeepTech2025/AgriX-AfricaDeepTech2025


In [57]:
# STEP 3: Upload image and auto-detect filename
from google.colab import files
import shutil
import os

uploaded = files.upload()
uploaded_filename = list(uploaded.keys())[0]

# Define destination path in your repo
destination_path = "/content/AgriX-AfricaDeepTech2025/data/crops/sample_leaf.jpg"

# Ensure destination folder exists
os.makedirs(os.path.dirname(destination_path), exist_ok=True)

# Move uploaded file to the destination
shutil.move(uploaded_filename, destination_path)

print(f"✅ Image uploaded and moved to: {destination_path}")


Saving sample_leaf.jpg to sample_leaf (1).jpg
✅ Image uploaded and moved to: /content/AgriX-AfricaDeepTech2025/data/crops/sample_leaf.jpg


In [59]:
prediction = predict_crop_disease(destination_path)
print(f"🌾 Predicted class: {prediction}")

🌾 Predicted class: Healthy


In [64]:
# STEP 5: Load the TFLite model
model_path = "/content/AgriX-AfricaDeepTech2025/prototype/tflite_model/crop_disease_model.tflite"
interpreter = tf.lite.Interpreter(model_path=model_path)
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print("📦 Model input shape:", input_details[0]['shape'])


📦 Model input shape: [  1 128 128   3]


In [65]:
# STEP 6: Preprocess image and run prediction
def predict_crop_disease(image_path):
    img = Image.open(image_path).convert('RGB').resize((128, 128))
    img_array = np.array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0).astype(np.float32)

    interpreter.set_tensor(input_details[0]['index'], img_array)
    interpreter.invoke()
    output = interpreter.get_tensor(output_details[0]['index'])[0]

    class_names = ['Healthy', 'Maize___Leaf_Spot', 'Tomato___Bacterial_spot']
    prediction = class_names[np.argmax(output)]
    confidence = np.max(output)

    return prediction, confidence


In [66]:
# STEP 7: Run the model on uploaded image
prediction, confidence = predict_crop_disease(LEAF_IMAGE_PATH)
print(f"🌾 Predicted class: {prediction} ({confidence*100:.2f}% confidence)")


🌾 Predicted class: Healthy (35.29% confidence)
